In [1]:
!pip install feedparser trafilatura transformers requests beautifulsoup4 pandas playwright yfinance

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 20.1 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=8f4b75ada400ba35a1631446fa7e15277c342a8261368a53f986757acb22bf74
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [2]:
import requests
import feedparser
import pandas as pd
import json
import time
import hashlib
import logging
import datetime as dt
from dataclasses import dataclass
from typing import List, Dict, Optional
from urllib.parse import urlparse
import trafilatura as tf
import re
from playwright.sync_api import sync_playwright
import numpy as np
import pandas as pd
import yfinance as yf
import feedparser
import requests
import pandas as pd
import time
from typing import List, Dict
import feedparser
import requests
import pandas as pd
import time
import trafilatura as tf
import re
from typing import List, Dict
from playwright.sync_api import sync_playwright
from bs4 import BeautifulSoup
import logging
import hashlib
import datetime as dt
from dataclasses import dataclass, field

In [ ]:


def collect_from_google_news(scraper_config: Dict) -> List[Dict]:
    """Collects articles from Google News RSS feeds."""
    articles = []
    keyword_groups = [
        "Apple OR AAPL OR Tim Cook",
        "iPhone OR iPad OR Apple Watch",
        "MacBook OR macOS OR Apple Silicon",
        "AirPods OR Vision Pro OR App Store"
    ]
    for group in keyword_groups:
        rss_url = f"https://news.google.com/rss/search?q={group.replace(' ', '+')}&hl=en-US&gl=US&ceid=US:en"
        feed = feedparser.parse(rss_url)
        for entry in feed.entries[:scraper_config['max_articles_per_source']]:
            articles.append({
                'url': entry.get('link', ''),
                'title': entry.get('title', ''),
                'source': 'Google News',
                'published_date': entry.get('published', '')
            })
        time.sleep(2)
    return articles

def collect_from_hacker_news(scraper_config: Dict) -> List[Dict]:
    """Collects articles from Hacker News API."""
    articles = []
    keywords = ["Apple", "AAPL", "iPhone", "MacBook", "Tim Cook", "iPad", "iOS", "macOS", "App Store", "Apple Watch", "AirPods", "M1 chip", "M2 chip", "Apple Silicon", "Vision Pro"]
    for keyword in keywords:
        api_url = f"https://hn.algolia.com/api/v1/search?query={keyword}&tags=story"
        try:
            response = requests.get(api_url, timeout=scraper_config['timeout'])
            if response.status_code == 200:
                data = response.json()
                for hit in data.get('hits', [])[:15]:
                    if hit.get('url') and hit.get('title'):
                        articles.append({
                            'url': hit['url'],
                            'title': hit['title'],
                            'source': 'Hacker News',
                            'published_date': hit.get('created_at', '')
                        })
                time.sleep(0.5)
        except Exception as error:
            continue
    return articles

def collect_from_tech_rss_feeds(scraper_config: Dict) -> List[Dict]:
    """Collects articles from a list of tech RSS feeds."""
    articles = []
    rss_feeds = [
        ("https://9to5mac.com/feed/", "9to5Mac"),
        ("https://techcrunch.com/tag/apple/feed/", "TechCrunch"),
        ("https://www.macrumors.com/macrumors.xml", "MacRumors"),
        ("https://appleinsider.com/rss/news", "AppleInsider"),
        ("https://daringfireball.net/feeds/main", "Daring Fireball"),
        ("https://www.apple.com/newsroom/rss-feed.rss", "Apple Official Newsroom"),
        ("https://www.cultofmac.com/feed", "Cult of Mac"),
        ("https://www.mactrast.com/feed/", "Mactrast"),
        ("http://feeds.feedburner.com/idropnews/feed", "iDrop News"),
        ("https://www.igeeksblog.com/feed/", "iGeeksBlog"),
        ("https://www.engadget.com/tag/apple/rss.xml", "Engadget Apple"),
        ("http://feeds.arstechnica.com/arstechnica/apple/", "Ars Technica Apple"),
        ("https://venturebeat.com/tag/apple/feed/", "VentureBeat Apple"),
    ]
    for rss_url, source_name in rss_feeds:
        try:
            feed = feedparser.parse(rss_url)
            for entry in feed.entries[:scraper_config['max_articles_per_source']]:
                articles.append({
                    'url': entry.get('link', ''),
                    'title': entry.get('title', ''),
                    'source': source_name,
                    'published_date': entry.get('published', '')
                })
            time.sleep(2)
        except Exception:
            continue
    return articles

def create_raw_dataframe(scraper_config: Dict) -> pd.DataFrame:
    """
    Orchestrates the scraping and returns a DataFrame of raw article data.
    """
    google_articles = collect_from_google_news(scraper_config)
    hacker_news_articles = collect_from_hacker_news(scraper_config)
    tech_rss_articles = collect_from_tech_rss_feeds(scraper_config)

    all_raw_articles = google_articles + hacker_news_articles + tech_rss_articles

    unique_articles = list({art['url']: art for art in all_raw_articles if art['url']}.values())

    df = pd.DataFrame(unique_articles)
    return df

if __name__ == '__main__':
    scraper_config = {
        'max_articles_per_source': 300,
        'timeout': 30
    }

    raw_df = create_raw_dataframe(scraper_config)
    print("DataFrame with raw scraped data:")
    print(raw_df.head())
    print(f"\nDataFrame shape: {raw_df.shape}")

DataFrame with raw scraped data:
                                                 url  \
0  https://news.google.com/rss/articles/CBMigwFBV...   
1  https://news.google.com/rss/articles/CBMilAFBV...   
2  https://news.google.com/rss/articles/CBMixwFBV...   
3  https://news.google.com/rss/articles/CBMitwFBV...   
4  https://news.google.com/rss/articles/CBMimwFBV...   

                                               title       source  \
0  'South Park' spoofs tech bros, Tim Cook, and T...  Google News   
1  'South Park' Mocks Tech Bros, Tim Cook, and Ch...  Google News   
2  South Park Blasts Mark Zuckerberg, Tim Cook fo...  Google News   
3  Apple increases U.S. commitment to $600 billio...  Google News   
4  Apple CEO Tim Cook’s $100 billion commitment t...  Google News   

                  published_date  
0  Sat, 23 Aug 2025 12:27:22 GMT  
1  Fri, 22 Aug 2025 02:09:00 GMT  
2  Thu, 21 Aug 2025 17:05:24 GMT  
3  Wed, 06 Aug 2025 07:00:00 GMT  
4  Thu, 07 Aug 2025 07:00:00 GMT  

Data

In [ ]:
raw_df

,url,title,source,published_date
0,https://news.google.com/rss/articles/CBMigwFBV...,"'South Park' spoofs tech bros, Tim Cook, and T...",Google News,"Sat, 23 Aug 2025 12:27:22 GMT"
1,https://news.google.com/rss/articles/CBMilAFBV...,"'South Park' Mocks Tech Bros, Tim Cook, and Ch...",Google News,"Fri, 22 Aug 2025 02:09:00 GMT"
2,https://news.google.com/rss/articles/CBMixwFBV...,"South Park Blasts Mark Zuckerberg, Tim Cook fo...",Google News,"Thu, 21 Aug 2025 17:05:24 GMT"
3,https://news.google.com/rss/articles/CBMitwFBV...,Apple increases U.S. commitment to $600 billio...,Google News,"Wed, 06 Aug 2025 07:00:00 GMT"
4,https://news.google.com/rss/articles/CBMimwFBV...,Apple CEO Tim Cook’s $100 billion commitment t...,Google News,"Thu, 07 Aug 2025 07:00:00 GMT"
...,...,...,...,...
1013,https://venturebeat.com/ai/inside-the-ai-integ...,Microsoft’s vs. Apple’s AI computer strategies...,VentureBeat Apple,"Thu, 23 May 2024 14:24:46 +0000"
1014,https://venturebeat.com/ai/ceding-control-how-...,Ceding control: How Copilot+ and PCs could mak...,VentureBeat Apple,"Wed, 22 May 2024 01:04:17 +0000"
1015,https://venturebeat.com/virtual/google-and-hp-...,Google and HP aim to bridge remote worker gap ...,VentureBeat Apple,"Mon, 13 May 2024 20:08:15 +0000"
1016,https://venturebeat.com/virtual/apple-slammed-...,Apple slammed by users over iPad Pro ‘Crush’ a...,VentureBeat Apple,"Wed, 08 May 2024 13:54:24 +0000"


In [ ]:



def start_our_logger():
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    return logging.getLogger(__name__)

logger = start_our_logger()

def clean_and_check_the_content(raw_content: str, min_length: int = 200) -> tuple[str, str]:
    if not raw_content:
        return "", "EMPTY"
    cleaned = raw_content.strip()
    cleaned = re.sub(r'\s+', ' ', cleaned)
    boilerplate_patterns = [
        r'Cookie Policy.*$', r'Privacy Policy.*$', r'Terms of Service.*$',
        r'Subscribe to.*newsletter.*$', r'Follow us on.*$'
    ]
    for pattern in boilerplate_patterns:
        cleaned = re.sub(pattern, '', cleaned, flags=re.IGNORECASE)
    if len(cleaned) < min_length:
        quality = "SHORT"
    elif len(cleaned) < 500:
        quality = "BASIC"
    elif len(cleaned) < 1500:
        quality = "GOOD"
    else:
        quality = "EXCELLENT"
    return cleaned.strip(), quality


import requests
import trafilatura as tf
import re
from playwright.sync_api import sync_playwright
from bs4 import BeautifulSoup
import logging
import time

def clean_and_check_the_content(raw_content: str, min_length: int = 200) -> tuple[str, str]:
    if not raw_content:
        return "", "EMPTY"
    cleaned = raw_content.strip()
    cleaned = re.sub(r'\s+', ' ', cleaned)
    boilerplate_patterns = [
        r'Cookie Policy.*$', r'Privacy Policy.*$', r'Terms of Service.*$',
        r'Subscribe to.*newsletter.*$', r'Follow us on.*$'
    ]
    for pattern in boilerplate_patterns:
        cleaned = re.sub(pattern, '', cleaned, flags=re.IGNORECASE)
    if len(cleaned) < min_length:
        quality = "SHORT"
    elif len(cleaned) < 500:
        quality = "BASIC"
    elif len(cleaned) < 1500:
        quality = "GOOD"
    else:
        quality = "EXCELLENT"
    return cleaned.strip(), quality

def get_the_article_text(url: str, headers: dict, timeout: int) -> tuple[str, str]:
    try:
        response = requests.get(url, headers=headers, timeout=timeout, allow_redirects=True)
        final_url = response.url
        if response.status_code == 200:
            html_content = response.text
            extracted_content = tf.extract(html_content, favor_precision=True)
            if extracted_content and len(extracted_content.strip()) > 100:
                clean_content, quality = clean_and_check_the_content(extracted_content)
                return clean_content, quality

        with sync_playwright() as p:
            browser = p.chromium.launch(headless=True)
            context = browser.new_context(user_agent=headers.get("User-Agent", ""))
            page = context.new_page()

            try:
                page.goto(url, timeout=timeout * 1000, wait_until="domcontentloaded")
                time.sleep(2)
                cookie_selectors = [
                    'button:has-text("Accept All")',
                    'button:has-text("Accept")',
                    '#consent-button',
                    '#cookie-accept-button',
                    'a:has-text("Accept")',
                    '#onetrust-accept-btn-handler'
                ]

                for selector in cookie_selectors:
                    try:
                        accept_button = page.locator(selector, has_text=re.compile("accept", re.IGNORECASE))
                        if accept_button.is_visible(timeout=5000):
                            accept_button.click()
                            time.sleep(1)
                            break
                    except Exception as e:
                        logging.info(f"Could not find or click selector {selector}: {e}")

                html_content_js = page.content()
                extracted_content = tf.extract(html_content_js, favor_precision=True) or ''

            except Exception as e:
                logging.error(f"Playwright navigation failed for {url}: {e}")
                extracted_content = ""

            finally:
                context.close()
                browser.close()

        clean_content, quality = clean_and_check_the_content(extracted_content)
        return clean_content, quality

    except requests.exceptions.Timeout:
        return "", "TIMEOUT"
    except requests.exceptions.RequestException:
        return "", "NETWORK_ERROR"
    except Exception as e:
        return "", f"ERROR: {str(e)[:50]}"

def collect_from_google_news(scraper_config: Dict) -> List[Dict]:
    articles = []
    keyword_groups = [
        "Apple OR AAPL OR Tim Cook",
        "iPhone OR iPad OR Apple Watch",
        "MacBook OR macOS OR Apple Silicon",
        "AirPods OR Vision Pro OR App Store"
    ]
    for group in keyword_groups:
        rss_url = f"https://news.google.com/rss/search?q={group.replace(' ', '+')}&hl=en-US&gl=US&ceid=US:en"
        feed = feedparser.parse(rss_url)
        for entry in feed.entries[:scraper_config['max_articles_per_source']]:
            articles.append({
                'url': entry.get('link', ''),
                'title': entry.get('title', ''),
                'source': 'Google News',
                'published_date': entry.get('published', '')
            })
        time.sleep(2)
    return articles

def collect_from_hacker_news(scraper_config: Dict) -> List[Dict]:
    articles = []
    keywords = ["Apple", "AAPL", "iPhone", "MacBook", "Tim Cook", "iPad", "iOS", "macOS", "App Store", "Apple Watch", "AirPods", "M1 chip", "M2 chip", "Apple Silicon", "Vision Pro"]
    for keyword in keywords:
        api_url = f"https://hn.algolia.com/api/v1/search?query={keyword}&tags=story"
        try:
            response = requests.get(api_url, timeout=scraper_config['timeout'])
            if response.status_code == 200:
                data = response.json()
                for hit in data.get('hits', [])[:15]:
                    if hit.get('url') and hit.get('title'):
                        articles.append({
                            'url': hit['url'],
                            'title': hit['title'],
                            'source': 'Hacker News',
                            'published_date': hit.get('created_at', '')
                        })
                time.sleep(0.5)
        except Exception:
            continue
    return articles

def collect_from_tech_rss_feeds(scraper_config: Dict) -> List[Dict]:
    articles = []
    rss_feeds = [
        ("https://9to5mac.com/feed/", "9to5Mac"),
        ("https://techcrunch.com/tag/apple/feed/", "TechCrunch"),
        ("https://www.macrumors.com/macrumors.xml", "MacRumors"),
        ("https://appleinsider.com/rss/news", "AppleInsider"),
        ("https://daringfireball.net/feeds/main", "Daring Fireball"),
        ("https://www.apple.com/newsroom/rss-feed.rss", "Apple Official Newsroom"),
        ("https://www.cultofmac.com/feed", "Cult of Mac"),
        ("https://www.mactrast.com/feed/", "Mactrast"),
        ("http://feeds.feedburner.com/idropnews/feed", "iDrop News"),
        ("https://www.igeeksblog.com/feed/", "iGeeksBlog"),
        ("https://www.engadget.com/tag/apple/rss.xml", "Engadget Apple"),
        ("http://feeds.arstechnica.com/arstechnica/apple/", "Ars Technica Apple"),
        ("https://venturebeat.com/tag/apple/feed/", "VentureBeat Apple"),
    ]
    for rss_url, source_name in rss_feeds:
        try:
            feed = feedparser.parse(rss_url)
            for entry in feed.entries[:scraper_config['max_articles_per_source']]:
                articles.append({
                    'url': entry.get('link', ''),
                    'title': entry.get('title', ''),
                    'source': source_name,
                    'published_date': entry.get('published', '')
                })
            time.sleep(2)
        except Exception:
            continue
    return articles

def create_raw_dataframe(scraper_config: Dict) -> pd.DataFrame:
    """
    Orchestrates the scraping, gets article content, and returns a DataFrame.
    """
    google_articles = collect_from_google_news(scraper_config)
    hacker_news_articles = collect_from_hacker_news(scraper_config)
    tech_rss_articles = collect_from_tech_rss_feeds(scraper_config)

    all_raw_articles = google_articles + hacker_news_articles + tech_rss_articles

    unique_urls = set()
    unique_articles = []
    for art in all_raw_articles:
        if art['url'] and art['url'] not in unique_urls:
            unique_articles.append(art)
            unique_urls.add(art['url'])

    processed_articles = []
    for article in unique_articles:
        print(f"Fetching content for: {article['url']}")
        content, _ = get_the_article_text(article['url'], scraper_config['headers'], scraper_config['timeout'])
        article['content'] = content
        processed_articles.append(article)
        time.sleep(2)

    df = pd.DataFrame(processed_articles)
    return df

if __name__ == '__main__':
    scraper_config = {
        'max_articles_per_source': 300,
        'timeout': 30,
        'headers': {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}
    }

    raw_df_with_content = create_raw_dataframe(scraper_config)
    print("DataFrame with raw scraped data and content:")
    print(raw_df_with_content.head())
    print(f"\nDataFrame shape: {raw_df_with_content.shape}")

Fetching content for: https://news.google.com/rss/articles/CBMigwFBVV95cUxOa1BwTWJ6MnZVcUJtcDFUeUJ2QlUxeWxpYkQ3MDNSY251S1RHYUhLZTJzRUdKWmFMdUJHM0JONmtOdlFmU1E5aUpPT09ZNXRROGhiVV9Uc21QZ3VWQThZeHRzYWt4WTF6QmNNcEdBajN4bkdsS285OVVqeE1jbXd6WXM3Yw?oc=5


Fetching content for: https://news.google.com/rss/articles/CBMilAFBVV95cUxQSkxrTUZHT19nclBZRmVKMzhxYkx0dlJfVVBaOHdOYUVMUDRRMzlXQmJsRWZxMjRKSmluRWZyNjZxcm1WNWpJYWVyNDdETVRBdzZISGNVMHN5dXZmOHN2WmVUWk9ZSmV0SVlaSWZrMzJ0aG5XWmJyM1ZHNEU4T1gwTmNndFFKOFV3LVloQ3B0NE16c2FR?oc=5


Fetching content for: https://news.google.com/rss/articles/CBMixwFBVV95cUxOa3VWLUt5dzhMSHc2bHVpQzFuS0wyNjl1MzZTUWZhZmtBeDFweFRiRHlSYmY5MUtsU3NacnBzYjJicDU4a1A3MkE5OFhGTXRTajh6cS1QUkxBTTd0VGRLMEVRRFpwOTd2Ym1SXzNoR3JpM2xXY2F1cjBPeVM3U1NLazFVVFJTbGtkUlZxNERlM0NrR0F4M1FleVUxVnN3MFFKYXk5ZTBYMnJfQzFucVRUSFFYZjBGVlhueWs3OUFYN2dWd3NIcjJv?oc=5


Fetching content for: https://news.google.com/rss/articles/CBMitwFBVV95cUxPUkdCcVdpRnpFR2xmWlNWUjBfWjJ5eERtdElrT1I3NXFHN25nX1E1TkJhTFRSZWw4QlU5MnpvYm94dTZ3bVpTTHpTaWF0dXRLN2JQQmRaM1cwSDl1NjZqNWM3MVdNcWVmMko1OGF4TnFKMU9RakIxZXk4UUFBU1FvMjBTX0FISDJaMzRvSHBxb0JqV0hnc3lvaGx0SHF5bzVvMGZERFFkVHhFSEdsU2FtMjJaVzNaUU0?oc=5


Fetching content for: https://news.google.com/rss/articles/CBMimwFBVV95cUxQa2ZyYUk5RUJxbGFNakZ0d1BKZENCaEthb0tWWGtvd3Y1VUVGeUJ1WmQ5cmxlX213X0IwZ0ZMRUNnbHpWR1NTR0lDRG1ta0l6X1JIa3FEU1NELWZjcUNYQ1JBYlI5OXhINlllS25VdlRudzVKOHBIT2l0cVhELW9DcHl3QW1NOWdRYzRjZEhqQkNBZENpZHBQTmF5cw?oc=5


...

Fetching content for: https://techcrunch.com/2024/03/06/apple-terminates-epic-games-developer-account-calling-it-a-threat-to-the-ios-ecosystem/
Fetching content for: https://www.reddit.com/r/apple/comments/782250/try_quickly_typing_1_2_3_into_the_ios_11/
Fetching content for: https://www.theverge.com/2019/1/31/18205795/apple-google-blocked-internal-ios-apps-developer-certificate
Fetching content for: https://9to5mac.com/2023/06/08/ios-17-link-tracking-protection/
Fetching content for: https://krausefx.com/blog/ios-privacy-stealpassword-easily-get-the-users-apple-id-password-just-by-asking
Fetching content for: https://www.theregister.co.uk/2019/10/02/apple_hong_kong/
Fetching content for: https://joreteg.com/blog/why-i-switched-to-android
Fetching content for: https://twitter.com/lemiorhan/status/935578694541770752
Fetching content for: https://saurabhs.org/advanced-macos-commands
Fetching content for: https://sigpipe.macromates.com/2020/macos-catalina-slow-by-design/
Fetching content 

Fetching content for: https://daringfireball.net/2020/12/heavy_is_the_head_that_wears_the_airpods_max
Fetching content for: https://www.engadget.com/a-robotics-engineer-is-trying-to-make-the-airpods-pro-easier-to-repair-184249580.html
Fetching content for: https://mariusmasalar.me/airpods-max
Fetching content for: https://blogs.gnome.org/jdressler/2024/06/26/do-a-firmware-update-for-your-airpods-now/
Fetching content for: https://techcrunch.com/2022/06/10/apple-m1-unpatchable-flaw/
Fetching content for: https://erik-engheim.medium.com/why-is-apples-m1-chip-so-fast-3262b158cba2
Fetching content for: https://www.zdnet.com/article/linux-6-2-the-first-mainstream-linux-kernel-for-apple-m1-chips-arrives/
Fetching content for: https://asia.nikkei.com/Business/Tech/Semiconductors/Apple-s-follow-up-to-M1-chip-goes-into-mass-production-for-Mac
Fetching content for: https://blog.yiningkarlli.com/2021/10/takua-on-m1-max.html
Fetching content for: https://www.apple.com/newsroom/2021/04/imac-feature

Fetching content for: https://kieranhealy.org/blog/archives/2025/06/28/american/
Fetching content for: https://daringfireball.net/2025/08/iphone_mirroring_more_than_one_iphone
Fetching content for: https://www.counterpointresearch.com/en/insights/post-insight-research-briefs-blogs-global-smart-glasses-shipments-soared-110-yoy-in-h1-2025-with-meta-capturing-over-70-share


Fetching content for: https://daringfireball.net/2025/08/joe_caroff_007_logo_designer
Fetching content for: https://dekaf.com/df
Fetching content for: https://yro.slashdot.org/comments.pl?sid=23765914&cid=65583466
Fetching content for: https://www.axios.com/2025/08/12/trump-bls-ej-antoni-economists
Fetching content for: https://www.theverge.com/news/695743/threads-dms-direct-messaging-launch
Fetching content for: https://www.gov.uk/government/news/national-drought-group-meets-to-address-nationally-significant-water-shortfall
Fetching content for: https://www.bloomberg.com/news/articles/2025-08-14/trump-administration-is-said-to-discuss-us-taking-stake-in-intel
Fetching content for: https://www.wsj.com/economy/the-u-s-marches-toward-state-capitalism-with-american-characteristics-f75cafa8?st=48pGHz&reflink=desktopwebshare_permalink
Fetching content for: https://x.com/backlon/status/1956075874370933114
Fetching content for: https://daringfireball.net/2025/08/apple_workaround_blood_oxygen_

Fetching content for: https://thecassetteapp.com/
Fetching content for: https://apnews.com/article/tiktok-bytedance-trump-perplexity-87988733973760927bb5681f7de9b9af
Fetching content for: https://www.wsj.com/tech/perplexity-ai-google-chrome-offer-5ddb7a22?st=r5gGet&reflink=desktopwebshare_permalink
Fetching content for: https://www.ft.com/content/cd1a0729-a8ab-41e1-a4d2-8907f4c01cac
Fetching content for: https://www.wsj.com/tech/nvidia-amd-chip-sales-us-government-f9e34b5f?st=pdZkXp&reflink=desktopwebshare_permalink
Fetching content for: https://truthsocial.com/@realDonaldTrump/posts/114987288040725570
Fetching content for: https://www.cnbc.com/2025/08/11/trump-china-tariffs-deadline-extended.html
Fetching content for: https://daringfireball.net/2025/08/openai_chatgpt_models_emotional_attachment
Fetching content for: https://www.tomshardware.com/service-providers/network-providers/aol-will-end-dial-up-internet-service-in-september-34-years-after-its-debut-aol-shield-browser-and-aol-dia

In [ ]:
raw_df_with_content.to_csv("/content/raw_df_with_content.csv",index=False)


In [49]:
raw_df_with_content = pd.read_csv("/content/raw_df_with_content.csv")

In [50]:
news = raw_df_with_content.copy()
news

,url,title,source,published_date,content
0,https://news.google.com/rss/articles/CBMigwFBV...,"'South Park' spoofs tech bros, Tim Cook, and T...",Google News,"Sat, 23 Aug 2025 12:27:22 GMT",NaN
1,https://news.google.com/rss/articles/CBMilAFBV...,"'South Park' Mocks Tech Bros, Tim Cook, and Ch...",Google News,"Fri, 22 Aug 2025 02:09:00 GMT",NaN
2,https://news.google.com/rss/articles/CBMixwFBV...,"South Park Blasts Mark Zuckerberg, Tim Cook fo...",Google News,"Thu, 21 Aug 2025 17:05:24 GMT",NaN
3,https://news.google.com/rss/articles/CBMitwFBV...,Apple increases U.S. commitment to $600 billio...,Google News,"Wed, 06 Aug 2025 07:00:00 GMT",NaN
4,https://news.google.com/rss/articles/CBMimwFBV...,Apple CEO Tim Cook’s $100 billion commitment t...,Google News,"Thu, 07 Aug 2025 07:00:00 GMT",NaN
...,...,...,...,...,...
1013,https://venturebeat.com/ai/inside-the-ai-integ...,Microsoft’s vs. Apple’s AI computer strategies...,VentureBeat Apple,"Thu, 23 May 2024 14:24:46 +0000",Want smarter insights in your inbox? Sign up f...
1014,https://venturebeat.com/ai/ceding-control-how-...,Ceding control: How Copilot+ and PCs could mak...,VentureBeat Apple,"Wed, 22 May 2024 01:04:17 +0000",Want smarter insights in your inbox? Sign up f...
1015,https://venturebeat.com/virtual/google-and-hp-...,Google and HP aim to bridge remote worker gap ...,VentureBeat Apple,"Mon, 13 May 2024 20:08:15 +0000",Google and HP today announced their intent to ...
1016,https://venturebeat.com/virtual/apple-slammed-...,Apple slammed by users over iPad Pro ‘Crush’ a...,VentureBeat Apple,"Wed, 08 May 2024 13:54:24 +0000","Apple, Inc. has had a rough year. But maybe it..."


In [52]:


news['published_dt'] = pd.to_datetime(
    news['published_date'],
    utc=True,
    errors='coerce',
    infer_datetime_format=True
)
news = news.dropna(subset=['published_dt'])


news['published_dt_only'] = news['published_dt'].dt.normalize()
news['published_dt_only'] = news['published_dt_only'].dt.tz_convert(None)


start_date = news['published_dt_only'].min()
end_date = news['published_dt_only'].max()
aapl = yf.download('AAPL', start=start_date, end=end_date + pd.Timedelta(days=1))


if isinstance(aapl.columns, pd.MultiIndex):
    aapl.columns = [
        col[0] if col[1] == '' else f"{col[0]}_{col[1]}"
        for col in aapl.columns.values
    ]

aapl = aapl.reset_index()
aapl['Date'] = pd.to_datetime(aapl['Date']).dt.normalize()


aapl = aapl.rename(columns={
    'Open': 'Open_AAPL',
    'High': 'High_AAPL',
    'Low': 'Low_AAPL',
    'Close': 'Close_AAPL',
    'Volume': 'Volume_AAPL'
})
cols_to_keep = ['Date', 'Open_AAPL', 'High_AAPL', 'Low_AAPL', 'Close_AAPL', 'Volume_AAPL']
aapl = aapl[[c for c in cols_to_keep if c in aapl.columns]]


merged_df = news.merge(
    aapl,
    left_on='published_dt_only',
    right_on='Date',
    how='left'
)

del merged_df['content']
del merged_df['published_dt']
del merged_df['Date']
del merged_df['published_date']

[*********************100%***********************]  1 of 1 completed


In [53]:
merged_df = merged_df.dropna(subset=['Open_AAPL']).reset_index(drop=True)

In [54]:
from transformers import pipeline
sentiment_analyzer = pipeline("sentiment-analysis")

merged_df['sentiment'] = merged_df['title'].apply(lambda x: sentiment_analyzer(str(x))[0]['label'])
merged_df['sentiment_score'] = merged_df['title'].apply(lambda x: sentiment_analyzer(str(x))[0]['score'])

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [55]:
merged_df.to_csv("/content/merged_df.csv",index=False)

In [94]:
merged_df.columns

Index(['url', 'title', 'source', 'published_dt_only', 'Open_AAPL', 'High_AAPL',
       'Low_AAPL', 'Close_AAPL', 'Volume_AAPL', 'sentiment', 'sentiment_score',
       'Volatility', 'Daily_Return', 'Sentiment_Lag1', 'Sentiment_Lag7',
       'Sentiment_Binary', 'Day_of_Week', 'Next_Day_Return',
       'Strategy_Return'],
      dtype='object')